In [1]:
# Enable env before running .\venv\Scripts\Activate
# Code for adding IDs for the players for each frame. The same player hasn't been assigned the same ID even after he leaves the frame yet. Re-ID will be done in day3
from tracker import Player, iou, compare_players
from ultralytics import YOLO
import cv2
import os
import json

In [ ]:
VIDEO_PATH = "15sec_input_720p.mp4"
MODEL_PATH = "best.pt"
CROPS_DIR = "players_crops"
FRAMES_DIR = "detected_frames_day2"

# Create folders if they don't exist
os.makedirs(CROPS_DIR, exist_ok=True)
os.makedirs(FRAMES_DIR, exist_ok=True)

# Load YOLO model
model = YOLO(MODEL_PATH)

# Open video
cap = cv2.VideoCapture(VIDEO_PATH)

# Initialize tracking variables
player_id_counter = 0
active_players = []
frame_id = 0
tracking_output = []


In [3]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    detections = []

    # Get new player detections
    for i, box in enumerate(results.boxes):
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])

        if model.names[cls_id] != "player" or conf < 0.5:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        crop = frame[y1:y2, x1:x2]

        detections.append({
            "bbox": [x1, y1, x2, y2],
            "crop": crop,
            "assigned": False
        })

    # Match new detections with existing players
    for det in detections:
        matched = False
        for player in active_players:
            if frame_id - player.last_seen > 5:
                continue  # ignore if player unseen for too long

            # Match based on IoU
            score = iou(det["bbox"], player.bbox)
            if score > 0.4:
                player.update(det["bbox"], frame_id, det["crop"])
                det["assigned"] = True
                matched = True
                tracking_output.append({
                    "frame": frame_id,
                    "id": player.id,
                    "bbox": det["bbox"]
                })
                break

        # Assign new ID if no match found
        if not matched:
            new_player = Player(player_id_counter, det["bbox"], frame_id, det["crop"])
            active_players.append(new_player)
            tracking_output.append({
                "frame": frame_id,
                "id": player_id_counter,
                "bbox": det["bbox"]
            })
            player_id_counter += 1

    # Draw IDs and save annotated frame
    for entry in tracking_output:
        if entry["frame"] == frame_id:
            x1, y1, x2, y2 = entry["bbox"]
            pid = entry["id"]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID: {pid}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    frame_save_path = f"{FRAMES_DIR}/frame_{frame_id:03}.jpg"
    cv2.imwrite(frame_save_path, frame)
    print(f"✅ Processed frame {frame_id}")
    frame_id += 1

cap.release()



0: 384x640 1 ball, 16 players, 2 referees, 18908.4ms
Speed: 241.3ms preprocess, 18908.4ms inference, 61.4ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed frame 0

0: 384x640 18 players, 2 referees, 21270.7ms
Speed: 248.0ms preprocess, 21270.7ms inference, 102.1ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed frame 1

0: 384x640 1 ball, 16 players, 2 referees, 17794.4ms
Speed: 15.5ms preprocess, 17794.4ms inference, 77.9ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed frame 2

0: 384x640 1 ball, 14 players, 2 referees, 17058.6ms
Speed: 48.9ms preprocess, 17058.6ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed frame 3

0: 384x640 1 ball, 14 players, 2 referees, 11870.0ms
Speed: 14.4ms preprocess, 11870.0ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed frame 4

0: 384x640 1 ball, 16 players, 2 referees, 12924.9ms
Speed: 49.4ms preprocess, 12924.9ms inference, 324.8ms postprocess per im

In [5]:
with open("player_tracks.json", "w") as f:
    json.dump(tracking_output, f, indent=2)

print(f"Saved tracking data for {len(tracking_output)} detections.")


Saved tracking data for 4780 detections.
